In [6]:
invisible(sapply(list.files("../rwwa", full.names = T), source))

return_periods <- list("tx14x" = 50, "tn14x" = 20)

# CMIP6

In [ ]:
varnm <- "tx14x"
nsamp <- 1000

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
rp <- return_periods[[varnm]]
for (fnm in list.files("ts", pattern = paste0(varnm,"_cmip6"), full.names = T)) {

    gcm <- paste0(strsplit(fnm, "_")[[1]][4:5], collapse = "_")   
    print(gcm)
    res_fnm <- gsub("ts/", "res/res-cmip6_", gsub(".dat", ".csv", fnm))
    if (file.exists(res_fnm)) next
    if(gcm %in% c("UKESM1-0-LL_r1i1p1f2")) next
     
    gmst_fnm <- list.files("ts", pattern = paste0("SGSAT.+", gcm), full.names = T)
    if(length(gmst_fnm) == 0) next
    if(length(gmst_fnm) > 1) gmst_fnm <- gmst_fnm[1] # in case multiple versions

    # load all the data
    ts <- load_ts(fnm, col.names = c("year", "x"))
    gmst <- load_ts(gmst_fnm, col.names = c("year", "gmst"))
    df <- merge(gmst, ts)
    df <- df[df$year <= 2070,]

    # pick up model to be fitted depending on variable
    mdl <- tryCatch({
            fit_ns("gev", "shift", data = df, varnm = "x", covnm = c("gmst"))
        }, error = function(cond) {
            return(NULL)
        })
    if (is.null(mdl)) { 
        mdl <- fit_ns("gev", "shift", data = df, varnm = "x", covnm = c("gmst"), method = "Nelder-Mead")
    }
    
    # use model's 2025 GMST to define factual climate        
    cov_now <- df[df$year == 2025,"gmst",drop = F]
    cov_hist <- cov_now - 1.3
    cov_fut <- cov_now + 1.3

    # bootstrap results
    res <- cmodel_results(mdl, rp = rp, cov_f = cov_now, cov_hist = cov_hist, cov_fut = cov_fut,
                          y_now = 2025, y_start = 1950, y_fut = 2070, nsamp = nsamp)
    res <- cbind(res, data.frame("ystart" = min(df$year), "yend" = max(df$year)))
    write.csv(res, res_fnm)

    # quick return level plots to check fit
    rlplot_fnm <- gsub("csv", "png", gsub("res/res", "mdl-eval/rlplot", res_fnm))
    if(!file.exists(rlplot_fnm)) {
        png(rlplot_fnm, h = 360, w = 480); par(cex.main = 1.4); {
            plot_returnlevels(mdl, cov_f = cov_now, cov_cf = cov_hist, nsamp = 100, main = gsub("ts/", "", gsub(".csv", "", fnm)))
        }; dev.off()
    }
}
print("Done.")

# CORDEX

In [105]:
varnm <- "tn14x"
nsamp <- 1000

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
rp <- switch(varnm, "tx14x" = 100, "tn14x" = 5)
for (fnm in list.files("ts", pattern = paste0(varnm,"_cordex"), full.names = T)) {

    gcm <- gsub("_", "_rcp85_", gsub("i1p1","",paste0(strsplit(fnm, "_")[[1]][4:5], collapse = "_")))
    mdl <- gsub(".dat","",paste0(strsplit(fnm, "_")[[1]][4:6], collapse = "_"))
    print(mdl)
    if (mdl %in% c("CNRM-CM5_r1i1p1_ALADIN53","HadGEM2-ES_r1i1p1_HadREM3-GA7-05")) next
                
    res_fnm <- gsub("ts/", "res/res-cordex_", gsub(".dat", ".csv", fnm))
    if (file.exists(res_fnm)) next
     
    gmst_fnm <- list.files("ts", pattern = paste0("smoothed-gsat_", gcm,".dat"), full.names = T)
    if(length(gmst_fnm) == 0) next

    # load all the data
    ts <- load_ts(fnm, col.names = c("year", "x"))
    gmst <- load_ts(gmst_fnm, col.names = c("year", "gmst"))
    df <- merge(gmst, ts)
    df <- df[df$year <= 2070,]

    # pick up model to be fitted depending on variable
    mdl <- tryCatch({
            fit_ns("gev", "shift", data = df, varnm = "x", covnm = c("gmst"))
        }, error = function(cond) {
            return(NULL)
        })
    if (is.null(mdl)) { 
        mdl <- fit_ns("gev", "shift", data = df, varnm = "x", covnm = c("gmst"), method = "Nelder-Mead")
    }
    
    # use model's 2025 GMST to define factual climate        
    cov_now <- df[df$year == 2025,"gmst",drop = F]
    cov_hist <- cov_now - 1.3
    cov_fut <- cov_now + 1.3

    # bootstrap results
    res <- cmodel_results(mdl, rp = rp, cov_f = cov_now, cov_hist = cov_hist, cov_fut = cov_fut,
                          y_now = 2025, y_start = 1950, y_fut = 2070, nsamp = nsamp)
    res <- cbind(res, data.frame("ystart" = min(df$year), "yend" = max(df$year)))
    write.csv(res, res_fnm)

    # quick return level plots to check fit
    rlplot_fnm <- gsub("csv", "png", gsub("res/res", "mdl-eval/rlplot", res_fnm))
    if(!file.exists(rlplot_fnm)) {
        png(rlplot_fnm, h = 360, w = 480); par(cex.main = 1.4); {
            plot_returnlevels(mdl, cov_f = cov_now, cov_cf = cov_hist, nsamp = 100, main = gsub("ts/", "", gsub(".csv", "", fnm)))
        }; dev.off()
    }
}
print("Done.")

[1] "CNRM-CM5_r1i1p1_ALADIN53"
[1] "CNRM-CM5_r1i1p1_ALADIN63"
[1] "CNRM-CM5_r1i1p1_CCLM4-8-17"
[1] "CNRM-CM5_r1i1p1_HadREM3-GA7-05"
[1] "CNRM-CM5_r1i1p1_HIRHAM5"
[1] "CNRM-CM5_r1i1p1_RACMO22E"
[1] "CNRM-CM5_r1i1p1_RCA4"
[1] "CNRM-CM5_r1i1p1_REMO2015"
[1] "EC-EARTH_r12i1p1_COSMO-crCLIM-v1-1"
[1] "EC-EARTH_r12i1p1_HadREM3-GA7-05"
[1] "EC-EARTH_r12i1p1_HIRHAM5"
[1] "EC-EARTH_r12i1p1_RACMO22E"
[1] "EC-EARTH_r12i1p1_RCA4"
[1] "EC-EARTH_r12i1p1_RegCM4-6"
[1] "EC-EARTH_r12i1p1_REMO2015"
[1] "EC-EARTH_r12i1p1_WRF361H"
[1] "EC-EARTH_r1i1p1_COSMO-crCLIM-v1-1"
[1] "EC-EARTH_r1i1p1_HIRHAM5"
[1] "EC-EARTH_r1i1p1_RACMO22E"
[1] "EC-EARTH_r1i1p1_RCA4"
[1] "EC-EARTH_r3i1p1_COSMO-crCLIM-v1-1"
[1] "EC-EARTH_r3i1p1_HIRHAM5"
[1] "EC-EARTH_r3i1p1_RACMO22E"
[1] "EC-EARTH_r3i1p1_RCA4"
[1] "HadGEM2-ES_r1i1p1_ALADIN63"
[1] "HadGEM2-ES_r1i1p1_CCLM4-8-17"
[1] "HadGEM2-ES_r1i1p1_COSMO-crCLIM-v1-1"
[1] "HadGEM2-ES_r1i1p1_HadREM3-GA7-05"
[1] "HadGEM2-ES_r1i1p1_HIRHAM5"
[1] "HadGEM2-ES_r1i1p1_RACMO22E"
[1] "HadGEM2-E

# Compile model res

In [106]:
varnm = "tn14x"
framing = "cordex"

mdl_res <- t(sapply(list.files("res", pattern = paste0(varnm,"_",framing), full.names = T), read.csv))
rownames(mdl_res) <- apply(sapply(strsplit(gsub(".csv","",rownames(mdl_res)), "_"), "[", 5:7), 2, paste0, collapse = "_")
write.csv(mdl_res, paste0("res-",framing,"_",varnm,".csv"))

ERROR: Error in loadNamespace(x): there is no package called ‘rwwa’
